## Task 2 - Is Rectangle?

### Task - Given four positive integers, determine if these four intergers taken as length of sides would form a rectangle.

Solution - Sides of rectangle have a property that parallel sides are equal in length. So for four arbitary intergers to form a rectangle two integers have to be equal to the remaining integers or they could all be euqal forming a square but that is also a case of a rectangle. 

Let the four positive integers be [a,b,c,d]. And 
$$f(x,y) =
  \begin{cases}
    1       & \quad \text{if } x=y \\
    0  & \quad \text{if } x\neq y 
  \end{cases}
$$  
be a equality check between x and y returning 0 is not equal and 1 if equal. We have 4 cases where the four numbers could form a rectangle. The cases are -

1. $a=b \text{ and } c=d$

2. $a=c \text{ and } b=d$

3. $a=d \text{ and } b=d$

4. $a=b=c=d$

We need to check for these cases, we have 6 equalities we have to check for, 6 comes from ${4 \choose 2}$. We can write the above cases in the form of product of equality checks -

1. $f(a,b)*f(c,d)$

2. $f(a,c)*f(b,d)$

3. $f(a,d)*f(b,c)$


Important to note that satisfying one of the checks corresponds to a rectangle.We need not check for the 4th case explicitly as we will have all the three checks to be 1 in the case of a square. 



#### Checking for equality using SWAP test

SWAP test is used to determine the overlap(inner product) between two qubits. Generally the inner product lies beetween 0 and 1, but in our case it is only 0 or 1. The circuit for a SWAP test between two qubits is as follows - 

In [25]:
import pennylane as qml

dev_swap = qml.device('default.qubit',wires=3)
@qml.qnode(dev_swap)
def swap_circ():

    qml.Hadamard(0)
    qml.ctrl(qml.SWAP([1,2]),control=0)
    qml.Hadamard(0)

    return qml.probs(0)
print(qml.draw(swap_circ)(),'\n Probabilites of the 0th qubit',swap_circ())

0: ──H─╭●─────H─┤  Probs
1: ────├SWAP────┤       
2: ────╰SWAP────┤        
 Probabilites of the 0th qubit [1. 0.]


From simple mathematics we can figure out the probability of $\ket{0}$ on the first qubits - $P(\ket{0}) = \frac{1}{2} + \frac{1}{2}|{\langle \psi \; , \;\phi \rangle}|^{2}$ . Here $\psi$ and $\phi$ are states that we want to compare. We can see from the above output the inner product if 1 as the probability of $\ket{0}$ is 1 because both states are $\ket{0}$. For our case the qubits will only be $\ket{0}$ or $\ket{1}$. This is because we use basis encoding to encode our intergers into the quantum circuit. Basis encoding encodes an interger $p$ into N qubits where $p\le 2^N -1$.  

#### SWAP test for N qubits states 

We used SWAP test on single qubits states but we can also use it in N qubit states, given that the input states are $\ket{x_1,x_2...x_N}$ where $x_i=0 \text{ or } 1$. If we have input states $\ket{x}=\ket{x_1,x_2,...x_N}$ and $\ket{y}=\ket{y_1,y_2,...x_N}$ where $x$ and $y$ are the integer representation of the input states. So, to demonstrate this I will go through the each step in N qubit SWAP test. 



Starting with quantum state  $\ket{0}\ket{x}\ket{y}$
1. Applying Hadmard to the 1st qubit 

$$(\frac{\ket{0}+\ket{1}}{\sqrt{2}})\ket{x}\ket{y}$$

2. Applying control SWAP gate to each of the qubit in $\ket{x}$ and $\ket{y}$

Sub-step 1 - Applying Control SWAP on 1st qubit

$$ \frac{\ket{0}\ket{x}\ket{y}+\ket{1}\ket{y_1,x_2,...x_N}\ket{x_1,y_2,...y_N}}{\sqrt{2}}$$

Sub-step i  - Applying Control SWAP on ith qubit

$$ \frac{\ket{0}\ket{x}\ket{y}+\ket{1}\ket{y_1,y_2,.,y_i,..x_N}\ket{x_1,x_2,.,x_i,..y_N}}{\sqrt{2}}$$


We keep this going till the Nth qubit.Sub-step N - Applying Control SWAP on Nth qubit.

$$ \frac{\ket{0}\ket{x}\ket{y}+\ket{1}\ket{y}\ket{x}}{\sqrt{2}}$$

3. Applying Hadmard to the 1st qubit.

Now if the $\ket{x}$ and $\ket{y}$ are equal then the Hadamrd will collapse the first qubit to $\ket{0}$ otherwise it will stay in $\ket{+}$.

$$H_0(\frac{\ket{0}\ket{x}\ket{y}+\ket{1}\ket{y}\ket{x}}{\sqrt{2}}) =
  \begin{cases}
    \ket{0}\ket{x}\ket{y}      & \quad \text{if } x=y \\
    \frac{\ket{0}\ket{x}\ket{y} + \ket{1}\ket{x}\ket{y} + \ket{0}\ket{y}\ket{x} - \ket{1}\ket{y}\ket{x}}{2}  & \quad \text{if } x\neq y 
  \end{cases}
$$  

So its clear that the probability for $\ket{0}$ will be 1 for $x=y$ and $\frac{1}{2}$ for $x\neq y$.

We can also chain this with other N qubits states. We can input 4 integers and compare a pairs of integers. \
For eg. If we input $[a,b,c,d]$ then we can compare let's say $f(a,c)$ and $f(b,d)$ which is one of the checks from above, and now if we measure the probability through P trials then the probability of $\ket{0}$ and we get $0$ then it means that $a=c \text{ and } b=d$ and the input forms a rectangle. 

In [24]:
import numpy as np
def find_n(feature:list)-> int: #Given a list returns N (the number of qubits required to represent the largest number in the list) 
    max_side = np.max(feature)
    binary = bin(max_side)[2:]
    return len(binary)

In [8]:
def is_rectangle(sides:list):
    #Defining number of qubits required to represent the largest number in the list 
    n_qubits = find_n(sides)
    
    #Defining measurement qubits
    measurement_qubits = range(3)
    n_measure = len(measurement_qubits)

    n_qubits_circ = n_measure+4*n_qubits

    dev_basis = qml.device('default.qubit',wires=n_qubits_circ+1,shots=100)

    #Defining the wire ranges for all our inputs
    wires_a = range(n_measure,n_qubits+n_measure)
    wires_b = range(n_qubits+n_measure,2*n_qubits+n_measure)
    wires_c = range(2*n_qubits+n_measure,3*n_qubits+n_measure)
    wires_d = range(3*n_qubits+n_measure,4*n_qubits+n_measure)

    # This is the SWAP operation which checks for equality
    def swap(wires_0,wires_1,control:int):
        #Comparing a and b - or rather checking if a == b - returns |0> on the ancilla if equal and returns |+>
        for i,j in zip(wires_0,wires_1):
            qml.ctrl(qml.SWAP([i,j]),control=control)

    @qml.qnode(dev_basis)
    def base_enc(feature:list):
        
        #Encoding our inputs into the circuit
        qml.BasisEmbedding(features=feature[0], wires=wires_a)
        qml.BasisEmbedding(features=feature[1], wires=wires_b)
        qml.BasisEmbedding(features=feature[2], wires=wires_c)
        qml.BasisEmbedding(features=feature[3], wires=wires_d)
        
        #SWAP test for the 1st case
        qml.Hadamard(measurement_qubits[0])
        swap(wires_a,wires_b,measurement_qubits[0])
        swap(wires_c,wires_d,measurement_qubits[0])
        qml.Hadamard(measurement_qubits[0])

        #SWAP test for the 2nd case
        qml.Hadamard(measurement_qubits[1])
        swap(wires_a,wires_c,measurement_qubits[1])
        swap(wires_b,wires_d,measurement_qubits[1])
        qml.Hadamard(measurement_qubits[1])

        #SWAP test for the 3rd case
        qml.Hadamard(measurement_qubits[2])
        swap(wires_a,wires_d,measurement_qubits[2])
        swap(wires_b,wires_c,measurement_qubits[2])
        qml.Hadamard(measurement_qubits[2])

        return qml.probs(measurement_qubits[0]),qml.probs(measurement_qubits[1]),qml.probs(measurement_qubits[2])
    

    #Priniting the circuit
    print(qml.drawer.draw(base_enc,max_length=120)(sides))

    #Storing the probabilities of the measurement qubits 
    circ_result = base_enc(sides)
    
    #Variable to Infer the result of the circuit 
    rectangle = 0

    
    for i,j in enumerate(circ_result):
        if  j[0]==1:
            rectangle = rectangle + 1
            
    if rectangle == 3 or rectangle == 1:
        return 1
    return 0



In [35]:

is_rectangle([6,3,3,5])

 0: ──H──────────────╭●────╭●────╭●────╭●────╭●────╭●─────H─────────────────────────────────────────────────────────
 1: ─────────────────│─────│─────│─────│─────│─────│──────H─╭●────╭●────╭●────╭●────╭●────╭●─────H──────────────────
 2: ─────────────────│─────│─────│─────│─────│─────│────────│─────│─────│─────│─────│─────│──────H─╭●────╭●────╭●───
 3: ─╭BasisEmbedding─├SWAP─│─────│─────│─────│─────│────────├SWAP─│─────│─────│─────│─────│────────├SWAP─│─────│────
 4: ─├BasisEmbedding─│─────├SWAP─│─────│─────│─────│────────│─────├SWAP─│─────│─────│─────│────────│─────├SWAP─│────
 5: ─╰BasisEmbedding─│─────│─────├SWAP─│─────│─────│────────│─────│─────├SWAP─│─────│─────│────────│─────│─────├SWAP
 6: ─╭BasisEmbedding─╰SWAP─│─────│─────│─────│─────│────────│─────│─────│─────├SWAP─│─────│────────│─────│─────│────
 7: ─├BasisEmbedding───────╰SWAP─│─────│─────│─────│────────│─────│─────│─────│─────├SWAP─│────────│─────│─────│────
 8: ─╰BasisEmbedding─────────────╰SWAP─│─────│─────│────────│───

0